### Instructions:  
Put session name in 'Session Name Here:' Field between the quotes, then click Kernel > Restart & Run All

Import libraries

In [ ]:
import statistics
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

Utility functions

In [ ]:
def split_list(func, cell_data):

    result = []

    temp = []

    for cell in cell_data:
        if func(cell):
            result.append(temp)
            temp = []
        else:
            temp.append(cell)

    return result

## Session Name Here:
Format: `YYYY-MM-DD-hh-mm-ss`

In [ ]:
session = '2019-07-10-16-28-25'

Open the session file and read its data:

In [ ]:
filename = 'log-'+session+'.csv'
file = open(filename,'r')
data = file.read()
file.close()

Format raw data from file so python can use it, and get all rotate events:

In [ ]:
data = list(map(lambda x: x.split(','), data.split('\n')))
clicks = [row for row in data if row[0]=='rotate']

## Time to solve the puzzle:

In [ ]:
try:
    timetosolve = int([row for row in data if row[0]=="solve"][0][1])-int([row for row in data if row[0]=="start"][0][1])
    print('Time to solve:',timetosolve/1000,'seconds')
except IndexError:
    print("Not solved.")

Get all hand turns, discard unneeded data

In [ ]:
turns = [(row[2],row[3],row[4]) for row in clicks]

Get list of hand co-ordinates

In [ ]:
%%capture 
coords = set()
[coords.add((n[0],n[1])) for n in turns]

## Click Stats:

In [ ]:
tot_redundant = 0
for n in coords:
    left = turns.count(n + tuple(['-1']))
    right = turns.count(n + tuple(['1']))
    redundant = sorted([left,right])[0]*2
    print(str(n),'-',redundant,'\n')
    tot_redundant += redundant

print('Total Clicks:         ',len(clicks))
print('Total Redundant turns:',tot_redundant)
print('        as % of total:','{:05.2f}'.format((tot_redundant*100)/len(clicks))+"%")

Process data for graphing:

In [ ]:
mouse_events = [row for row in data if row[0]=='mouse']
mouse_events_x = list(map(int,[row[2] for row in mouse_events]))
mouse_events_y = list(map(int,[row[3] for row in mouse_events]))

click_events = [row for row in data if row[0]=='click']
click_events_x = list(map(int,[row[2] for row in click_events]))
click_events_y = list(map(int,[row[3] for row in click_events]))

button0 = [(int(row[2]),int(row[3])) for row in data if row[0]=='button0_pos' or row[0]=='button0_size']
button1 = [(int(row[2]),int(row[3])) for row in data if row[0]=='button1_pos' or row[0]=='button1_size']
button2 = [(int(row[2]),int(row[3])) for row in data if row[0]=='button2_pos' or row[0]=='button2_size']
button3 = [(int(row[2]),int(row[3])) for row in data if row[0]=='button3_pos' or row[0]=='button3_size']
canvas = [(int(row[2]),int(row[3])) for row in data if row[0]=='canvas_pos' or row[0]=='canvas_size']
hand00 = [(canvas[0][0],canvas[0][1]),        (250,             250)]
hand01 = [(canvas[0][0],canvas[0][1]+250),    (250,             canvas[1][1]-250)]
hand10 = [(canvas[0][0]+250,canvas[0][1]),    (250,             250)]
hand11 = [(canvas[0][0]+250,canvas[0][1]+250),(250,             canvas[1][1]-250)]
hand20 = [(canvas[0][0]+500,canvas[0][1]),    (canvas[1][0]-500,250)]
hand21 = [(canvas[0][0]+500,canvas[0][1]+250),(canvas[1][0]-500,canvas[1][1]-250)]

items = [button0,button1,button2,button3,canvas,hand00,hand01,hand10,hand11,hand20,hand21]
item_plots_x = [[item[0][0],item[0][0]+item[1][0],item[0][0]+item[1][0],item[0][0],item[0][0]] for item in items]
item_plots_y = [[item[0][1],item[0][1],item[0][1]+item[1][1],item[0][1]+item[1][1],item[0][1]] for item in items]

win_dimensions = [[row[2],row[3]] for row in data if row[0]=='win_dimensions']

func = lambda x: x==-1
mouse_events_x = split_list(func, mouse_events_x)
mouse_events_y = split_list(func, mouse_events_y)

## Mouse movement graph:

In [ ]:
plt.figure(dpi=2400)
for index in range(len(item_plots_x)):
    plt.plot(item_plots_x[index], item_plots_y[index], '-k',linewidth=0.1)
for index in range(len(mouse_events_x)):
    plt.plot(mouse_events_x[index], mouse_events_y[index], '-or',linewidth=0.25,markersize=0.75)
plt.plot(click_events_x,click_events_y,'ob',markersize=2)
plt.axis([0, int(win_dimensions[0][0]), 0, int(win_dimensions[0][1])])
plt.xticks(range(0,int(win_dimensions[0][0]),100))
plt.yticks(range(0,int(win_dimensions[0][1]),100))
plt.gca().invert_yaxis()
plt.show()

Red:   Mouse movement, dots every 25ms (1/40s)  
Blue:  Mouse Clicks  
Black: UI Elements  